<img align="right" src="../../additional_data/banner_siegel.png" style="width:1100px;">

# Data Lookup and Loading

* [**Sign up to the JupyterHub**](https://www.phenocube.org/) to run this notebook interactively from your browser
* **Compatibility:** Notebook currently compatible with the Open Data Cube environments of the University of Wuerzburg
* **Products used**: `s2_l2a_bavaria`
* **Prerequisites**:  Users of this notebook should have a basic understanding of:
    * How to run a [Jupyter notebook](01_jupyter_introduction.ipynb)
    * The basic structure of the eo2cube [satellite datasets](02_eo2cube.ipynb)

## Background

A "datacube" is a digital information architecture that specialises in hosting and cataloguing spatial information. Eo2cube is based on the [Open Data Cube](https://www.opendatacube.org/) infrastructure and specialises in storing remotely sensed data, particularly from Earth Observation satellites such as Landsat and Sentinel-2. The eo2cube contains multiple analyse ready satellite data "products". These data products are often composed of a range of "measurements" such as the suite of remote sensing band values. This notebook will focus on several straightforward ways to inspect the product,  measurement of a datacube, as well as to run data query and loading the data using a wide range of options, including resampling and interpolation.

## Description

This notebook demonstrates how to **browse the available products/ measurements** stored within, and how to **load data** from the eo2cube datacube by using the `dc.load()` function. Topics covered include:

* How to connect to the datacube
* How to list all the products/ measurements
* Loading data with the `dc.load()`function
* Customising the `dc.load()`query
* Tips and tricks to simplify the data loading process

***

## Load packages

The `datacube` package is required to query the eo2cube datacube database and load the requested data. The pandas package is required to format tables. The `with_ui_cbk` function from `odc.ui` enables a progress bar when loading large amounts of data

In [2]:
import datacube
import pandas as pd
from odc.ui import with_ui_cbk
sys.path.append("../../../../Scripts")
from DEAPlotting import display_map

# Set some configurations for displaying tables nicely
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', None)

## Datacube connection

After importing the datacube package, you are ready to connect to the eo2cube datacube. Users need to specify a name for their session, known as the app name. This name is generated by the user and is used to track down issues with database queries. It does not have any effect on the analysis. Use a short name that is consistent with the purpose of your notebook such as the way 03_Products_and_measurements has been used as the app name in this notebook.

The resulting dc object provides access to all the data contained within the eo2cube datacube.

In [3]:
dc = datacube.Datacube(app = '03_Products_and_measurement', config = '/home/datacube/.datacube.conf') #datacube connection

## List products

Once a datacube instance has been created, users can explore the products and measurements stored within.
The following cell lists all products that are currently available in the eo2cube datacube by using the dc.list_products() function.

In [4]:
dc.list_products()

,name,description,format,creation_time,platform,lat,label,instrument,product_type,time,lon,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,,,
44,S1_GRD_index,Gamma Naught RTC backscatter,GTiff,None,SENTINEL-1,None,None,C-SAR,gamma0,None,None,NaN,NaN,NaN,NaN
39,dsm1sv10,DSM 1sec Version 1.0,NETCDF,None,Space Shuttle Endeavour,None,None,SIR,DEM,None,None,NaN,NaN,NaN,NaN
40,dsm1sv10_burkinafaso,Elevation data from the Shuttle Radar Topography Mission.,NetCDF,None,Space Shuttle Endeavour,None,None,SIR,DEM,None,None,EPSG:4326,"(0.00027777777776258203, 0.00027777777776258203)","(0.333332761149, 0.333332761149)","(latitude, longitude)"
42,gpm_imerg_gis_daily,"GIS translation of the imerg data product. Multi satellite, global, precipitation estimate. Downloaded from ftp://arthurhou.pps.eosdis.nasa.gov/gpmdata/. Rate measured over one day.",GeoTiff,None,GPM,None,None,GPM,daily,None,None,NaN,NaN,NaN,NaN
43,gpm_imerg_gis_daily_global,Global NetCDF GPM IMERG GIS data,NetCDF,None,GPM,None,None,GPM,daily,None,None,EPSG:4326,"(-0.1, 0.1)","(90, 180)","(latitude, longitude)"
14,ls5_c1_ard_scene,Landsat 5 USGS Collection 1 Higher Level SR scene proessed using LEDAPS. 30m UTM based projection.,GeoTiff,None,LANDSAT_5,None,None,TM,LEDAPS,None,None,NaN,NaN,NaN,NaN
1,ls5_collections_sr_scene,Landsat 5 USGS Collection 1 Higher Level SR scene proessed using LEDAPS. 30m UTM based projection.,GeoTiff,None,LANDSAT_5,None,None,TM,LEDAPS,None,None,NaN,NaN,NaN,NaN
6,ls5_ledaps_demmin,Landsat 5 USGS Collection 1 Higher Level SR scene proessed using LEDAPS. Resampled to 30m EPSG:4326 projection with a sub degree tile size.,NetCDF,None,LANDSAT_5,None,None,TM,LEDAPS,None,None,EPSG:4326,"(-0.000269494585236, 0.000269494585236)","(0.943231048326, 0.943231048326)","(latitude, longitude)"
17,ls7_c1_ard_scene,Landsat 7 USGS Collection 1 Higher Level SR scene processed using LEDAPS. 30m UTM based projection.,GeoTiff,None,LANDSAT_7,None,None,ETM,LEDAPS,None,None,NaN,NaN,NaN,NaN


## List measurements

Every product is associated with a range of available measurements. For the most time, these are the individual satellite bands.

The dc.list_measurements() function can be used to interrogate the measurements associated with a given product (specified by the name column from the table above). For example, s2_l2a_bavaria refers to the Sentinel 2 ARD L2A scenes product for Bavaria.

In [6]:
product = "s2_l2a_bavaria"

measurements = dc.list_measurements()
measurements.loc[product]

,name,dtype,units,nodata,aliases,flags_definition
measurement,,,,,,
coastal_aerosol,coastal_aerosol,int16,reflectance,0,"[band_1, coastal_aerosol]",NaN
blue,blue,int16,reflectance,0,"[band_2, blue]",NaN
green,green,int16,reflectance,0,"[band_3, green]",NaN
red,red,int16,reflectance,0,"[band_4, red]",NaN
red_edge1,red_edge1,int16,reflectance,0,"[band_5, veg5]",NaN
red_edge2,red_edge2,int16,reflectance,0,"[band_6, veg6]",NaN
red_edge3,red_edge3,int16,reflectance,0,"[band_7, veg7]",NaN
nir,nir,int16,reflectance,0,"[band_8, nir]",NaN
narrow_nir,narrow_nir,int16,reflectance,0,"[band_8a, narrow_nir]",NaN


## Display Map
Before loading the data, it is a good idea to first check out the area of interest in a map. We can use display_map function for that. The following code confirm our coordinates correspond to our area of interest Würzburg, Germany. 

**Don't forget to import function `display_map` from `DEAPlotting` when you load the libraries.**

In [3]:
display_map([49.7352601, 49.8335172],[9.8506165, 10.0538635], crs = 'EPSG:4326')

/home/datacube/anaconda3/envs/jupyterhub/lib/python3.9/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/datacube/anaconda3/envs/jupyterhub/lib/python3.9/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/home/datacube/anaconda3/envs/jupyterhub/lib/python3.9/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>

## Loading data with the `dc.load()`function

Loading data from the datacube uses the `dc.load()` function.

The function requires the following minimum arguments:

* product: The data product to load (to see all available eo2cube products, see the [Products and measurements notebook](03_products_and_measurements.ipynb)).
* x: The spatial region in the x dimension. By default, the x and y arguments accept queries in a geographical co-ordinate system WGS84, identified by the EPSG code 4326.
* y: The spatial region in the y dimension. The dimensions longitude/latitude and x/y can be used interchangeably. It is also possible to use the extent of an imported shapefile as x/y (see the [notebook for working with vectordata](XX_vectordata.ipynb)).
* time: The temporal extent. The time dimension can be specified using a tuple of datetime objects or strings in the “YYYY”, “YYYY-MM” or “YYYY-MM-DD” format.

This example loads all Sentinel-2 data from the first half of April 2020 for an area around Würzburg. The used product is defined by its name. The spatial extent is defined by lat/lon coordinates. The time intervall ist defined by the format "YYYY-MM-DD". The argument `progess_cbk` with the imported `with_ui_cbk()`function enables loading bar.

In [13]:
data = dc.load(product= "s2_l2a_bavaria",
               x= (9.8506165, 10.0538635),
               y= (49.7352601, 49.8335172),
               time= ("2020-04-01", "2020-04-15"),
               progress_cbk=with_ui_cbk())

data

<xarray.Dataset>
Dimensions:          (time: 6, x: 1477, y: 1112)
Coordinates:
  * time             (time) datetime64[ns] 2020-04-01T10:26:57 ... 2020-04-14...
  * y                (y) float64 5.521e+06 5.521e+06 ... 5.51e+06 5.51e+06
  * x                (x) float64 5.612e+05 5.612e+05 ... 5.759e+05 5.759e+05
    spatial_ref      int32 25832
Data variables:
    coastal_aerosol  (time, y, x) int16 408 475 475 475 475 475 ... 0 0 0 0 0 0
    blue             (time, y, x) int16 412 389 451 527 637 790 ... 0 0 0 0 0 0
    green            (time, y, x) int16 760 695 703 829 947 1140 ... 0 0 0 0 0 0
    red              (time, y, x) int16 620 599 708 909 1042 1262 ... 0 0 0 0 0
    red_edge1        (time, y, x) int16 1366 1356 1356 1522 1522 ... 0 0 0 0 0
    red_edge2        (time, y, x) int16 3188 2814 2814 2466 2466 ... 0 0 0 0 0
    red_edge3        (time, y, x) int16 3656 3138 3138 2742 2742 ... 0 0 0 0 0
    nir              (time, y, x) int16 3894 3363 3162 3064 3022 ... 0 0 0 0 0
    narrow_nir       (time, y, x) int16 3883 3476 3476 3071 3071 ... 0 0 0 0 0
    water_vapour     (time, y, x) int16 3824 3392 3392 3392 3392 ... 0 0 0 0 0
    swir1            (time, y, x) int16 2329 2494 2494 2816 2816 ... 0 0 0 0 0
    swir2            (time, y, x) int16 1299 1461 1461 1836 1836 ... 0 0 0 0 0
    scl              (time, y, x) uint8 4 4 4 4 4 5 5 5 5 ... 0 0 0 0 0 0 0 0 0
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

## Customization of the `dc.load()` function

The query created above only used the basic arguments of the `dc.load()` function. However, it is possible to customize your loading query by using multiple other arguments. A detailed description of every useable argument can be found in the [original documentation of the `dc.load()` function](https://datacube-core.readthedocs.io/en/latest/dev/api/generate/datacube.Datacube.load.html) of the Opendatacube initiative. In this notebook we will present the most used arguments that allow easy and effective querying of eo2cube products.

### Specifying Measurements
<a id='first_query'></a>
The `measurements` argument provides the option to filter the desired product by its bands. This argument takes a list of the names of the desired measurement, as listed in the `dc.list_measurements()` function (see [notebook 03](03_products_and_measurements.ipynb)). If not provided, all stored measurements of the product will be loaded.

In the following code only the `blue`, `green` and `red` bands of the `s2_l2a_bavaria` product are added to the query. The resulting dataset will only include the defined measurements.

In [28]:
data = dc.load(product= "s2_l2a_bavaria",
               measurements= ["blue", "green", "red"],
               x= (9.8506165, 10.0538635),
               y= (49.7352601, 49.8335172),
               time= ("2020-04-01", "2020-04-15"),
               progress_cbk=with_ui_cbk())

data

<xarray.Dataset>
Dimensions:      (time: 6, x: 1477, y: 1112)
Coordinates:
  * time         (time) datetime64[ns] 2020-04-01T10:26:57 ... 2020-04-14T10:...
  * y            (y) float64 5.521e+06 5.521e+06 5.521e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.612e+05 5.612e+05 ... 5.759e+05 5.759e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 412 389 451 527 637 790 703 ... 0 0 0 0 0 0
    green        (time, y, x) int16 760 695 703 829 947 1140 ... 0 0 0 0 0 0
    red          (time, y, x) int16 620 599 708 909 1042 1262 ... 0 0 0 0 0 0
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

### Specifying CRS and Reprojecting 

By default, the x and y coordinates of the coordinate reference system (CRS) of the query are always assumed to be `WGS84 / EPSG:4326`. If these coordinates are in a different coordinate system, it is necessary to use the `crs` argument.

To reproject the data from the default or stored CRS into a specified CRS, it is neccessary to define the `output_crs` and a new `resolution` for your data. The unit of the new `output_crs` and `resolution`must identical.

In this example code, the `s2_l2a_bavaria` product (with default CRS EPSG:25832) is transformed into the `output_crs` WGS84/EPSG:4326 with a suitable `resolution` of (-0.0002734477121776, 0.0002734477121776) arc degrees. Note that the dimensions and coordinates changed from y/x to latitude/longitude. Due to the larger resolution the absolute number of pixel in the dataset is lower. Also the `crs` variable changed to `EPSG:4326`.

In [24]:
data = dc.load(product= "s2_l2a_bavaria",
               measurements= ["blue", "green", "red"],
               x= (9.8506165, 10.0538635),
               y= (49.7352601, 49.8335172),
               time= ("2020-04-01", "2020-04-15"),
               output_crs = "EPSG:4326",
               resolution = (-0.0002734477121776, 0.0002734477121776),
               progress_cbk=with_ui_cbk())

data

<xarray.Dataset>
Dimensions:      (latitude: 360, longitude: 744, time: 6)
Coordinates:
  * time         (time) datetime64[ns] 2020-04-01T10:26:57 ... 2020-04-14T10:...
  * latitude     (latitude) float64 49.83 49.83 49.83 ... 49.74 49.74 49.74
  * longitude    (longitude) float64 9.851 9.851 9.851 ... 10.05 10.05 10.05
    spatial_ref  int32 4326
Data variables:
    blue         (time, latitude, longitude) int16 614 422 411 412 ... 0 0 0 0
    green        (time, latitude, longitude) int16 1024 770 694 699 ... 0 0 0 0
    red          (time, latitude, longitude) int16 1114 727 673 684 ... 0 0 0 0
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

### Spatial resampling

To resample the data to another resolution, multiple resampling methods are available. You can define the `resampling` argument with one method from the following:
`"nearest", "cubic", "bilinear", "cubic_spline", "lanczos", "average", "mode", "gauss", "max", "min", "med", "q1", "q3"`.
By default, the `dc.load()` function is using "nearest neighbour" resampling, which allocates each new pixel with the value of the closest input pixel. However, for some data (eg. continuous data) this may not be the most appropiate choice.

The example code below, resamples the `s2_l2a_bavaria` product to a greater resolution of (-20, 20) meters using the bilinear interpolation method (`bilinear`). Note that the absolute number of pixels has been halved as the dataset has been resampled to twice the pixel size ([compare with the first query](#first_query))

In [31]:
data = dc.load(product= "s2_l2a_bavaria",
               measurements= ["blue", "green", "red"],
               x= (9.8506165, 10.0538635),
               y= (49.7352601, 49.8335172),
               time= ("2020-04-01", "2020-04-15"),
               resolution = (-20, 20),
               resampling = "bilinear",
               progress_cbk=with_ui_cbk())

data

<xarray.Dataset>
Dimensions:      (time: 6, x: 739, y: 556)
Coordinates:
  * time         (time) datetime64[ns] 2020-04-01T10:26:57 ... 2020-04-14T10:...
  * y            (y) float64 5.521e+06 5.521e+06 5.521e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.612e+05 5.612e+05 ... 5.759e+05 5.759e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 408 439 566 818 668 350 343 ... 0 0 0 0 0 0
    green        (time, y, x) int16 754 752 860 1106 1064 555 ... 0 0 0 0 0 0
    red          (time, y, x) int16 574 705 930 1294 1236 558 ... 0 0 0 0 0 0
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

It is also possible to define different resampling methods for different measurements. This is useful if a dataset stores continuous data and categorical data. For example, the `s2_l2a_bavaria` has a scene classification layer (`scl`) which contains categorical values about the quality of a single pixel. If you want to resample your continuous data to another resolution with the `bilinear` method but also include the categorical `scl` band (categorical) it is best to use an interpolation method that does not modify the data directly for the resampling of the categorical `scl` band. 

This can be done by using Python Dictionaries as value for the `resampling` argument. The following code resample continuous bands with `bilinear` interpolation but also uses the `nearest` (which does not modify the data values) for the categorical `scl` band.

In [29]:
data = dc.load(product= "s2_l2a_bavaria",
               measurements= ["blue", "green", "red", "scl"],
               x= (9.8506165, 10.0538635),
               y= (49.7352601, 49.8335172),
               time= ("2020-04-01", "2020-04-15"),
               resolution = (-20, 20),
               resampling = {"*": "bilinear", "scl": "nearest"},
               progress_cbk=with_ui_cbk())

data

<xarray.Dataset>
Dimensions:      (time: 6, x: 739, y: 556)
Coordinates:
  * time         (time) datetime64[ns] 2020-04-01T10:26:57 ... 2020-04-14T10:...
  * y            (y) float64 5.521e+06 5.521e+06 5.521e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.612e+05 5.612e+05 ... 5.759e+05 5.759e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 408 439 566 818 668 350 343 ... 0 0 0 0 0 0
    green        (time, y, x) int16 754 752 860 1106 1064 555 ... 0 0 0 0 0 0
    red          (time, y, x) int16 574 705 930 1294 1236 558 ... 0 0 0 0 0 0
    scl          (time, y, x) uint8 4 4 4 5 5 4 4 4 4 4 ... 0 0 0 0 0 0 0 0 0 0
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

### Group by argument

The `group_by` argument is a useful tool to combine/mosaic multiple scenes of one date (e.g. if the extent is visible in two sequential scenes of the satellite). Satellite datasets can have multiple observations per day with slightly different time stamps as the satellite collects data along its path. These observations can be combined by reducing the time dimension to the day level using `group_by = "solar_day"`. By default the scenes are simply copied together (e.g. the NA values of the first scene are filled with the values of the second scene). With the argument "fuse_func" the mosaicing can be controlled by the user (e.g. specify an interpolation method).

This example loads data for the first week in April 2020 from the `s2_l2a_bavaria` for a bigger area in Frankonia. Since the extent of the AOI is to big to be covered by just one Sentinel 2 scene there are multiple scenes (from contiguous paths) available for each date. By using the `group_by = "solar_day"` argument the `dc.load()` function automatically mosaics all scenes of a single date together. In this example no special interpolation method was defined.

In [34]:
data = dc.load(product= "s2_l2a_bavaria",
               measurements= ["blue", "green", "red"],
               x= (9.8506165, 11.273325),
               y= (49.7352601, 50.191334),
               time= ("2020-04-01", "2020-04-07"),
               group_by = "solar_day",
               progress_cbk=with_ui_cbk())

data

<xarray.Dataset>
Dimensions:      (time: 4, x: 10310, y: 5284)
Coordinates:
  * time         (time) datetime64[ns] 2020-04-01T10:26:54 ... 2020-04-06T10:...
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.607e+05 5.607e+05 ... 6.638e+05 6.638e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 493 302 271 355 265 ... 322 323 238 202 258
    green        (time, y, x) int16 628 394 401 498 405 ... 479 429 322 311 367
    red          (time, y, x) int16 805 459 438 600 436 ... 581 512 409 380 421
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

### Loading data `like` another dataset

Another very useful tool to load data from the eo2cube environment is the `like` argument. With the `like` argument it is possible to load a desired product exactly like another stored product. This will copy the spatial and temporal extent, the CRS and resolution from an existing dataset.

In this example the `s2_l2a_mwpomerania` product will be loaded by defining the query manually. The second dataset, `ls8_lasrc_demmin` which covers the same area in Northern Germany but was recoreded by Landsat 8, is then loaded `like` the first `s2_data` dataset. The defined extent (`x`, `y`) and `time` intervall is copied. Also the default `crs` and `resolution` of the `s2_data` dataset is used to reproject and resample the `ls8_lasrc_demmin` product. Only the desired bands of the `ls8_lasrc_demmin` and if necessary the `group_by` argument need to be defined.

In [67]:
s2_data = dc.load(product= "s2_l2a_mwpomerania",
                   measurements= ["blue", "green", "red", "scl"],
                   x= (12.994152, 13.086261),
                   y= (53.873347, 53.938069),
                   time= ("2019-02-01", "2019-02-28"),
                   group_by = "solar_day",
                   progress_cbk=with_ui_cbk())

s2_data

<xarray.Dataset>
Dimensions:      (time: 12, x: 647, y: 755)
Coordinates:
  * time         (time) datetime64[ns] 2019-02-01T10:25:56 ... 2019-02-28T10:...
  * y            (y) float64 5.984e+06 5.984e+06 ... 5.977e+06 5.977e+06
  * x            (x) float64 7.621e+05 7.621e+05 ... 7.686e+05 7.686e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 5196 5324 5416 5332 ... 11152 11464 11432
    green        (time, y, x) int16 4920 4932 4948 4992 ... 9744 10016 10256
    red          (time, y, x) int16 4908 4860 4968 4988 ... 8376 8688 9120 9408
    scl          (time, y, x) uint8 9 9 9 9 9 9 9 9 9 9 ... 9 9 9 9 9 9 9 9 9 9
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

In [69]:
ls8_data = dc.load(product = "ls8_lasrc_demmin",
                 like = s2_data,
                 measurements = ["blue", "green", "red", "pixel_qa"],
                 group_by = "solar_day",
                 progress_cbk=with_ui_cbk())

ls8_data

<xarray.Dataset>
Dimensions:      (time: 2, x: 647, y: 755)
Coordinates:
  * time         (time) datetime64[ns] 2019-02-16T10:01:53 2019-02-23T10:08:02
  * y            (y) float64 5.984e+06 5.984e+06 ... 5.977e+06 5.977e+06
  * x            (x) float64 7.621e+05 7.621e+05 ... 7.686e+05 7.686e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 294 278 278 303 303 ... 201 201 201 201 141
    green        (time, y, x) int16 522 526 526 549 549 ... 362 362 362 362 294
    red          (time, y, x) int16 443 445 445 485 485 ... 320 320 320 320 267
    pixel_qa     (time, y, x) int32 322 322 322 322 322 ... 322 322 322 322 322
Attributes:
    crs:           PROJCS["ETRS89 / UTM zone 32N",GEOGCS["ETRS89",DATUM["Euro...
    grid_mapping:  spatial_ref

## Recommended next steps

To continue with the beginner's guide, the following notebooks are designed to be worked through in the following order:

1. [Jupyter Notebooks](01_jupyter_introduction.ipynb)
2. [eo2cube](02_eo2cube.ipynb)
3. **Products and Measurements (this notebook)**
4. [Loading data](04_loading_data.ipynb)
5. [Advanced xarrays operations](05_advanced_xarray.ipynb)
6. [Plotting data](06_plotting.ipynb)
7. [Basic analysis of remote sensing data](07_basic_analysis.ipynb)
8. [Parallel processing with Dask](08_parallel_processing_with_dask.ipynb)

***

## Additional information

<font size="2">This notebook for the usage in the Open Data Cube entities of the [Department of Remote Sensing](http://remote-sensing.org/), [University of Wuerzburg](https://www.uni-wuerzburg.de/startseite/), is adapted from [Geoscience Australia](https://github.com/GeoscienceAustralia/dea-notebooks), published using the Apache License, Version 2.0. Thanks! </font>

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.


**Contact:** If you would like to report an issue with this notebook, you can file one on [Github](https://github.com).

**Last modified:** January 2021